In [2]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to select the .zip file

Saving Dataset3.zip to Dataset3.zip


In [3]:
import zipfile
import os

with zipfile.ZipFile("Dataset3.zip", 'r') as zip_ref:
    zip_ref.extractall("Dataset3")


In [3]:
import os
from google.colab import files
import zipfile

# 1. Upload your folder (as a ZIP file)
uploaded = files.upload()

# 2. Unzip the uploaded file
for file_name in uploaded.keys():
    if file_name.endswith(".zip"):
        with zipfile.ZipFile(file_name, 'r') as zip_ref:
            zip_ref.extractall("uploaded_folder")
        print(f"Extracted: {file_name}")

# 3. Walk through the folder and show its structure
def show_directory_structure(folder_path):
    for dirpath, dirnames, filenames in os.walk(folder_path):
        level = dirpath.replace(folder_path, '').count(os.sep)
        indent = ' ' * 4 * level
        print(f'{indent}{os.path.basename(dirpath)}/')
        subindent = ' ' * 4 * (level + 1)
        for f in filenames:
            print(f'{subindent}{f}')

# 4. Show structure of the extracted folder
show_directory_structure("uploaded_folder")


KeyboardInterrupt: 

In [4]:
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf

# Load training dataset (grayscale, smaller size)
training_dataset = image_dataset_from_directory(
    directory='Dataset3/Dataset3',
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',              # Use grayscale
    batch_size=32,
    image_size=(128, 128),               # Reduced image size
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    verbose=True
)

# Load validation dataset
validation_dataset = image_dataset_from_directory(
    directory='Dataset3/Dataset3',
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    verbose=True
)

# Normalize pixel values to [0, 1]
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

training_dataset = training_dataset.map(normalize)
validation_dataset = validation_dataset.map(normalize)

Found 1200 files belonging to 6 classes.
Using 960 files for training.
Found 1200 files belonging to 6 classes.
Using 240 files for validation.


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
model = Sequential([
    Input(shape=(128, 128, 1)),  # Grayscale input (1 channel)

    # Block 1
    Conv2D(16, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    # Block 2
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    # Block 3
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Flatten(),
    Dense(128, activation='relu'),  # Reduced size
    Dropout(0.3),
    Dense(26, activation='softmax')  # 26 classes
])

optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(training_dataset,
                   epochs=15,
                  callbacks=[early_stop],
                   validation_data=validation_dataset)

Epoch 1/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 15s 112ms/step - accuracy: 0.6888 - loss: 2.2889 - val_accuracy: 0.1667 - val_loss: 2.5262
Epoch 2/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9932 - loss: 0.0180 - val_accuracy: 0.1833 - val_loss: 2.1494
Epoch 3/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9967 - loss: 0.0168 - val_accuracy: 0.2375 - val_loss: 1.9986
Epoch 4/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.9992 - loss: 0.0019 - val_accuracy: 0.5500 - val_loss: 1.7170
Epoch 5/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 0.7167 - val_loss: 1.3582
Epoch 6/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.9989 - loss: 0.0045 - val_accuracy: 0.8000 - val_loss: 0.9322
Epoch 7/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9976 - loss: 0.0055 - val_accuracy: 0.8250 - val_loss: 0.5545
Epoch 8/15
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9995 - loss: 0.0026 - val_accuracy: 0.9417 -

In [6]:
from google.colab import files

model.save('model1111.keras')
files.download('model1111.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>